# Convert Voice Memos into Project Plan
The following code accepts voice memos as input and turn them into project plans.

### Import necessary libraries
If you do not have these installed then you can install them using `pip install` command

In [ ]:
import openai
import os
import shutil
import glob
import json
from datetime import datetime

### Get a list of all the mp4 files to process
Let's start by getting list of all the audio files.
File uploads are currently limited to 25MB and the following input file types are supported: mp3, mp4, mpeg, mpga, m4a, wav, and webm.
If the filesize is greather than 25MB then the file will be ignored.

In [ ]:
import os
import glob

folder_path = "voice memos"  # Replace with the path to your "voice memos" folder

# Define a list of accepted file extensions
accepted_formats = ["mp3", "mp4", "mpeg", "mpga", "m4a", "wav", "webm"]

# Use glob to find all files with the accepted extensions in the folder
audio_files = []
for accepted_format in accepted_formats:
    pattern = os.path.join(folder_path, f"*.{accepted_format}")
    audio_files.extend(glob.glob(pattern))

# Print the list of matching file paths
for audio_file in audio_files:
    file_size = os.path.getsize(audio_file)
    # Check if the file size is greater than 25MB (25 * 1024 * 1024 bytes)
    if file_size > 25 * 1024 * 1024:
        print(f"File {audio_file} is larger than 25MB and will be ignored.")
        audio_files.remove(audio_file)
    else:
        print(audio_file)
        print(audio_files)

### Get audio transcripts
Get transcipt from the audio files using whisper API

In [ ]:
# Function to transcribe audio files
def transcribe_audio_files(audio_files):
    transcripts = []
    for audio_file in audio_files:
        # Construct the full path to the audio file
        audio_path = audio_file

        # Read the audio file as binary
        with open(audio_path, "rb") as audio_:
            try:
                # Transcribe the audio using OpenAI
                print(f"Transcibing {audio_file}")
                transcript = openai.Audio.transcribe("whisper-1", audio_)
                transcripts.append(transcript)
                print(f"Transcibtion Complete for {audio_file}")
            except Exception as e:
                transcripts.append(f"Transcription error: {str(e)}")

    return transcripts

# Get transcripts for the list of audio files
transcripts = transcribe_audio_files(audio_files)

# Print the transcripts
for i, transcript in enumerate(transcripts):
    print(f"Transcript {i + 1}:")
    print(transcript)
    print("\n")

# You can also save the transcripts to a file or perform further processing as needed


### Prompt to convert voice memo to project plan
Helper function to create prompt to convert transcript to a project plan

In [ ]:
def create_prompt_for_transcript(transcript):
    prompt = f"""Convert the following memo transcript, which is delimited by ``` in the variable 'memo_transcript', \
    into a detailed project plan. The project plan should include the following elements:
    
    1. Project Title: [Provide a suitable project title]
    2. Project Objective: [Clearly state the project's objective]
    3. Project Scope: [Define the scope of the project]
    4. Deliverables: [List all the project deliverables]
    5. Milestones: [Identify key project milestones and their timelines]
    6. Resources: [Specify the required resources, including personnel, equipment, and budget]
    7. Tasks and Activities: [Break down the project into individual tasks and activities]
    8. Dependencies: [Identify task dependencies and relationships]
    9. Timeline: [Create a project timeline or schedule]
    10. Risks and Mitigations: [List potential risks and their mitigation strategies]
    11. Communication Plan: [Outline how project stakeholders will be informed and involved]
    
    Please ensure that the resulting project plan is well-structured, organized, and comprehensive. You may need to \
    summarize and reformat the information from the memo transcript to create a clear and actionable project plan.
    
    memo_transcript:
    ```
    {transcript}
    ```
    """
    
    return prompt

### Convert transcripts into project plans
Run through the transcipts and create project plans by calling OpenAI Completion API

In [ ]:
# Initialize the list to store responses
responses = []
i = 0
# Loop through the transcripts and call OpenAI API
for transcript in transcripts:
    i += 1
    print(f"Creating project plan for Transcript {i}")
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        temperature=0,
        messages=[
            {"role": "user", "content": create_prompt_for_transcript(transcript)}
        ]
    )['choices'][0]['message']['content']
    
    print(f"Project Plan for Transcript {i}: \n{response}")
    
    responses.append(response)

# Now, the 'responses' list contains the completions for each transcript
print(responses)

### Append the extracted data to json file stored on hard drive
This function stores transcript and project plan in a json file for future use.

In [ ]:
# Extract just the file names without the directory path
file_names = [os.path.basename(file) for file in audio_files]

print(file_names)

# Create a list of dictionaries with transcript, file name, and response
result_list = []
current_time = datetime.now().isoformat()

for transcript, file_name, response in zip(transcripts, file_names, responses):
    result = {
        "transcript": transcript,
        "file_name": file_name,
        "response": response,
        "time": current_time
    }
    result_list.append(result)

# Append the data to the projects.json file
output_filename = "projects.json"

# Check if the file exists
if os.path.exists(output_filename):
    with open(output_filename, "r") as file:
        existing_data = json.load(file)
else:
    existing_data = []

existing_data.extend(result_list)

# Write the combined data to the projects.json file
with open(output_filename, "w") as file:
    json.dump(existing_data, file, indent=4)

print(f"Data appended to {output_filename}")


### Move the files in the list audio_file from the voice memo folder to processed.
Remove the processed file from the working folder (voice memo) to processed.

In [ ]:
# Destination directory
destination_dir = "processed"

# Ensure the destination directory exists, create it if it doesn't
if not os.path.exists(destination_dir):
    os.makedirs(destination_dir)

# Loop through the list of MP4 files and move them to the processed folder
for file in audio_files:
    source_path = file
    destination_path = os.path.join(destination_dir, os.path.basename(file))

    # Check if the file exists in the source directory before moving
    if os.path.exists(source_path):
        shutil.move(source_path, destination_path)
        print(f"Moved {file} to {destination_dir}")
    else:
        print(f"File {file} does not exist in the source directory")

# Remove the files in the "voice memos" directory
for root, dirs, files in os.walk("voice memos"):
    for file in files:
        file_path = os.path.join(root, file)
        os.remove(file_path)
